<a href="https://colab.research.google.com/github/ImitatedSky/stock/blob/main/stock_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
stock = pd.read_csv('stock50.csv')
stock.head()

def normalize(df):
    norm = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    return norm



In [ ]:
stock = stock.dropna()
stock['y'] = stock['Close'].shift(-1)
stock.iloc[:,2:4] = normalize(stock.iloc[:,2:4])
stock.head()

In [ ]:
def train_windows(df, ref_day=5, predict_day=1):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day+1):
        X_train.append(np.array(df.iloc[i:i+ref_day,:-1]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day]["y"]))
    return np.array(X_train), np.array(Y_train)

In [ ]:
class stock_lstm(keras.Model):
  def __init__(self,units):
      super(stock_lstm, self).__init__()
      self.rnn = Sequential([
          layers.SimpleRNN(units,input_shape=(5,2),dropout=0.2,return_sequences=True,unroll=True),
          layers.SimpleRNN(units,dropout=0.2,unroll=True)])
      self.fc = layers.Dense(32)
      self.out = layers.Dense(1)
  def call(self, inputs, training=None):
      x = self.rnn(inputs)
      x = self.fc(x)
      out = self.out(x)
      return out

In [ ]:
rnn_model = stock_lstm(units)
rnn_model.compile(optimizer  =keras.optimizers.Adam(),loss="mean_absolute_error",metrics=['mean_absolute_error'])
rnn_model.fit(data,epochs=epochs, validation_data = data_test,shuffle=True)